In [73]:
import os
print(os.getcwd())

/workspace/code/gym-continuousDoubleAuction/gym_continuousDoubleAuction


In [74]:
# %cd ..

In [75]:
# !pip freeze > requirements.txt

In [76]:
# !ls -l

In [77]:
# %cd gym_continuousDoubleAuction

In [78]:
# !pwd

In [79]:
%run ../gym_continuousDoubleAuction/test/test_accounting.py

.............
----------------------------------------------------------------------
Ran 13 tests in 0.008s

OK


In [80]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_new.py

............
----------------------------------------------------------------------
Ran 12 tests in 0.006s

OK


In [81]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_double_delete_order.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [82]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_volume_sync.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [83]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_crossed_book.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [84]:
%run ../gym_continuousDoubleAuction/test/test_modify_order.py

......
----------------------------------------------------------------------
Ran 6 tests in 0.003s

OK


In [85]:
# %run ../gym_continuousDoubleAuction/test/test_nav_callback.py

In [86]:
%run ../gym_continuousDoubleAuction/test/test_new_action_space.py

........
----------------------------------------------------------------------
Ran 8 tests in 0.026s

OK


In [87]:
%run ../gym_continuousDoubleAuction/inspect_latest_episode.py

Latest file found: ../gym_continuousDoubleAuction/episode_data/f5887d99e77f47b6ab7bb55ef30bfada.pkl

Data successfully written to latest_episode_data.txt


In [88]:
%run ../gym_continuousDoubleAuction/inspect_latest_episode_json.py

Latest file found: ../gym_continuousDoubleAuction/episode_data/f5887d99e77f47b6ab7bb55ef30bfada.pkl

Data successfully written to latest_episode_data.json


In [89]:
# %run ../gym_continuousDoubleAuction/analyze_unused.py

In [90]:
from datetime import datetime

now = datetime.now()
formatted = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted)  # e.g., "2025-12-22 10:30:45"

2025-12-24 13:32:44
